<a href="https://colab.research.google.com/github/RylieWeaver/NLP/blob/main/Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 7.4 Lab Assignment #4: Create Pre-trained BERT Vectors

https://colab.research.google.com/drive/1iisDj2kU-P6UIYeMZnT58ZGLdA_fDwNw?usp=sharing

## Setup

### Mount Drive

In [4]:
# Setup access to my Google Drive
from google.colab import drive
drive.mount('/content/drive',  force_remount=True)

Mounted at /content/drive


In [5]:
# Travel to Lab2 Directory
%cd "/content/drive/MyDrive/NLP/Lab4"

/content/drive/MyDrive/NLP/Lab4


In [6]:
%ls

bert_split.csv        bert_truncated.csv        longformer.csv      Restaurant_reviews_R.csv
bert_split_table.png  bert_truncated_table.png  longform_table.png


### Pip

In [4]:
# If needed
!pip install transformers

ERROR: Operation cancelled by user


### General Imports

In [7]:
import numpy as np
import pandas as pd
import torch

## Section 1

### Read File

In [8]:
column_names = ['Rating', 'Date', 'Review', 'Business', 'City', 'State', 'Zip', 'Business Rating Score']
bert_truncated_reviews = pd.read_csv('Restaurant_reviews_R.csv', header=None, names=column_names)

In [9]:
bert_truncated_reviews.head()

,Rating,Date,Review,Business,City,State,Zip,Business Rating Score
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.0
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.5


In [17]:
bert_truncated_reviews.shape

(63230, 8)

### Import Model

In [13]:
from transformers import BertTokenizer, BertModel

In [14]:
import logging
# Set logging level to ERROR to mute informational messages
logging.getLogger("transformers").setLevel(logging.ERROR)

In [15]:
# Show time to run this
%%time
# Load the pre-trained base bert tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# Load pre-trained model
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

CPU times: user 1.03 s, sys: 956 ms, total: 1.98 s
Wall time: 5.95 s


### Preprocessing

Base BERT requires minimal preprocessing because BERT is trained to work with raw text

Investigate if truncation is necessary (this is not necessary since we have the truncate is a function option, but is interesting to investigate)

In [19]:
# Tokenizing the review texts
%%time
token_len = pd.DataFrame()
token_len['token_ids'] = bert_truncated_reviews['Review'].apply(
    lambda x: tokenizer.encode(
        x,
        truncation=False,
        add_special_tokens=True
        )
    )

CPU times: user 3min 31s, sys: 495 ms, total: 3min 32s
Wall time: 4min 13s


In [20]:
token_len['tokenized_reviews'] = token_len['token_ids'].apply(
lambda ids: tokenizer.convert_ids_to_tokens(ids))

In [21]:
token_len['token_count'] = token_len['tokenized_reviews'].apply(len)

In [22]:
long_text = token_len[token_len['token_count'] > 512].sort_values(
by='token_count', ascending=False)
print(len(long_text))

1099


In [23]:
long_text['token_count']

25529    1332
14400    1326
43725    1321
21213    1279
33140    1273
         ... 
28349     513
59967     513
64        513
19831     513
15642     513
Name: token_count, Length: 1099, dtype: int64

In [24]:
print(token_len.shape)
print(long_text.shape)

(63230, 3)
(1099, 3)


Actual tokenization with truncation. I redo the operation so as not to create the image that the previous section was not necessary to truncate

In [30]:
# Tokenizing the review texts
%%time
bert_truncated_reviews['token_ids'] = bert_truncated_reviews['Review'].apply(
    lambda x: tokenizer.encode(
        x,
        truncation=True,
        add_special_tokens=True
        )
    )

CPU times: user 3min 49s, sys: 812 ms, total: 3min 50s
Wall time: 4min 29s


In [31]:
bert_truncated_reviews['tokenized_reviews'] = bert_truncated_reviews['token_ids'].apply(
lambda ids: tokenizer.convert_ids_to_tokens(ids))

In [34]:
bert_truncated_reviews['token_count'] = bert_truncated_reviews['tokenized_reviews'].apply(len)

In [35]:
bert_truncated_reviews.head()

,Rating,Date,Review,Business,City,State,Zip,Business Rating Score,token_ids,tokenized_reviews,token_count
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,"[101, 2049, 1996, 2190, 10733, 12570, 1045, 10...","[[CLS], its, the, best, pizza, hut, i, ', v, f...",50
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,"[101, 2023, 2173, 2003, 2019, 5875, 25025, 101...","[[CLS], this, place, is, an, interesting, comb...",143
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,"[101, 10733, 12570, 2003, 2307, 999, 2017, 213...","[[CLS], pizza, hut, is, great, !, you, get, hu...",59
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.0,"[101, 2467, 1037, 2307, 2173, 2000, 5510, 2070...","[[CLS], always, a, great, place, to, taste, so...",23
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.5,"[101, 3972, 4509, 999, 3030, 1999, 2006, 1037,...","[[CLS], del, ##ish, !, stopped, in, on, a, ran...",86


In [38]:
# Check truncation (these should all give 512)
print(bert_truncated_reviews['token_count'].loc[[25529, 14400, 43725, 64, 19831, 15642]])


"""Some of the truncated review tokenizations"""
# Index    Length
# 25529     1332
# 14400     1326
# 43725     1321
#           ...
# 64         513
# 19831      513
# 15642      513

25529    512
14400    512
43725    512
64       512
19831    512
15642    512
Name: token_count, dtype: int64


'Some of the truncated review tokenizations'

### Computation Setup

In [19]:
# Check if GPU is available and set computational device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [20]:
# Move model to the selected device
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [21]:
# Enforce that the model is in evaluation mode
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

### Generate Embeddings

In [22]:
# tqdm gives progress bars
from tqdm import tqdm

In [23]:
def generate_embeddings(df, batch_size=16):
    embeddings = [] # To store embeddings

    # Process in batches
    for i in tqdm(range(0, len(df), batch_size)):
        batch = df[i:i+batch_size]

        # Make inputs
        inputs = tokenizer(
            batch.tolist(),  # batch must be as list
            padding=True,  # pad
            truncation=True,  # truncate
            add_special_tokens=True,  # common default that holds here
            return_tensors="pt",  # pytorch
            max_length=512,  # max token length
            return_attention_mask=True  # use attention masks
            )

        # Get output from model/input
        inputs = {k: v.to(device) for k, v in inputs.items()}  # put inputs on the same device as the model
        with torch.no_grad():  # no gradients in inference mode
          outputs = model(**inputs)

        batch_embeddings = outputs.pooler_output.detach().cpu().numpy()  # pool output
        embeddings.append(batch_embeddings)  # append embeddings for batch

    embeddings = np.concatenate(embeddings, axis=0)  # concatenate all batch embeddings
    return embeddings

In [24]:
# Make embeddings in batches and observe time
%%time
embeddings = generate_embeddings(bert_truncated_reviews['Review'])

100%|██████████| 3952/3952 [28:14<00:00,  2.33it/s]

CPU times: user 26min 13s, sys: 5.39 s, total: 26min 19s
Wall time: 28min 14s


### Make Final DataFrame and Show

In [27]:
# Attach embeddings
bert_truncated_reviews['Base BERT Vectors'] = list(embeddings)

In [28]:
bert_truncated_reviews.head()

,Rating,Date,Review,Business,City,State,Zip,Business Rating Score,Base BERT Vectors
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,"[-0.66720617, -0.5012724, -0.97912174, 0.69763..."
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,"[-0.58872235, -0.5280975, -0.9803678, 0.500642..."
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,"[-0.854242, -0.44665197, -0.9462228, 0.7120740..."
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.0,"[-0.729353, -0.29508904, -0.89169145, 0.598858..."
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.5,"[-0.7442665, -0.47785082, -0.988199, 0.6841982..."


In [29]:
bert_truncated_reviews.shape

(63230, 9)

### Save DataFrame

Put together dataframe and embeddings

In [30]:
%pwd

'/content/drive/MyDrive/NLP/Lab4'

In [31]:
%ls

Restaurant_reviews_R.csv


In [32]:
bert_truncated_reviews.to_csv('bert_truncated.csv', index=False)

## Section 2

### Read File

In [5]:
column_names = ['Rating', 'Date', 'Review', 'Business', 'City', 'State', 'Zip', 'Business Rating Score']
longformer_reviews = pd.read_csv('Restaurant_reviews_R.csv', header=None, names=column_names)

In [6]:
longformer_reviews.head()

,Rating,Date,Review,Business,City,State,Zip,Business Rating Score
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.0
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.5


In [7]:
longformer_reviews.shape

(63230, 8)

### Import Model

In [8]:
from transformers import LongformerTokenizer, LongformerModel

In [9]:
import logging
# Set logging level to ERROR to mute informational messages
logging.getLogger("transformers").setLevel(logging.ERROR)

In [10]:
# Show time to run this
%%time
# pre-trained tokenizer
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
# pre-trained model
model = LongformerModel.from_pretrained('allenai/longformer-base-4096')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

CPU times: user 1.38 s, sys: 1.83 s, total: 3.21 s
Wall time: 8.39 s


### Computation Setup

In [11]:
# Check if GPU is available and set computational device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [12]:
# Move model to the selected device
model.to(device)

LongformerModel(
  (embeddings): LongformerEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (position_embeddings): Embedding(4098, 768, padding_idx=1)
  )
  (encoder): LongformerEncoder(
    (layer): ModuleList(
      (0-11): 12 x LongformerLayer(
        (attention): LongformerAttention(
          (self): LongformerSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (query_global): Linear(in_features=768, out_features=768, bias=True)
            (key_global): Linear(in_features=768, out_features=768, bias=True)
            (value_global): Linear(in_features=768, out_features=768, bias=True)
          )
    

In [13]:
# Enforce that the model is in evaluation mode
model.eval()

LongformerModel(
  (embeddings): LongformerEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (position_embeddings): Embedding(4098, 768, padding_idx=1)
  )
  (encoder): LongformerEncoder(
    (layer): ModuleList(
      (0-11): 12 x LongformerLayer(
        (attention): LongformerAttention(
          (self): LongformerSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (query_global): Linear(in_features=768, out_features=768, bias=True)
            (key_global): Linear(in_features=768, out_features=768, bias=True)
            (value_global): Linear(in_features=768, out_features=768, bias=True)
          )
    

### Preprocessing (N/A)

Longformer also requires minimal preprocessing. Compared to base BERT, it has it's own tokenizer and a global attention mask.

Also, longformer can process sequences of up to 4096 tokens, which is more than the largest review as we saw in section 1. Thus, there are no reviews that exceed the token maximum, and we don't need preprocessing

### Generate Embeddings

In [14]:
# For progress bars
from tqdm import tqdm

In [15]:
def generate_longformer_embeddings(df, batch_size=16):
    embeddings = [] # To store embeddings

    # Process in batches
    for i in tqdm(range(0, len(df), batch_size)):
        batch = df[i:i+batch_size]

        # Make inputs
        inputs = tokenizer(
            batch.tolist(),  # batch must be as list
            padding=True,  # pad
            truncation=True,  # truncate
            add_special_tokens=True,  # common default that holds here
            return_tensors="pt",  # pytorch
            max_length=512,  # max token length
            return_attention_mask=True  # use attention masks
            )

        # Create a global attention mask for longformer
        global_attention_mask = torch.zeros_like(inputs['input_ids'])  # Same shape as the input_ids
        global_attention_mask[:, 0] = 1  # Set global attention on the first token ([CLS] token)

        # Get output from model/input
        inputs = {k: v.to(device) for k, v in inputs.items()}  # put inputs on the same device as the model
        with torch.no_grad():  # no gradients in inference mode
          outputs = model(**inputs)

        batch_embeddings = outputs.pooler_output.detach().cpu().numpy()  # pool output
        embeddings.append(batch_embeddings)  # append embeddings for batch

    embeddings = np.concatenate(embeddings, axis=0)  # concatenate all batch embeddings
    return embeddings

In [16]:
# Make embeddings in batches and observe time
%%time
embeddings = generate_longformer_embeddings(longformer_reviews['Review'])

100%|██████████| 3952/3952 [49:35<00:00,  1.33it/s]

CPU times: user 47min 57s, sys: 6.38 s, total: 48min 4s
Wall time: 49min 35s


### Make Final DataFrame and Show

In [17]:
# Attach embeddings
longformer_reviews['Longform Vectors'] = list(embeddings)

In [18]:
longformer_reviews.head()

,Rating,Date,Review,Business,City,State,Zip,Business Rating Score,Longform Vectors
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,"[0.18431816, -0.24602777, 0.058935184, 0.00836..."
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,"[0.18255672, -0.32250398, 0.08609148, -0.00297..."
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,"[0.18899634, -0.30677307, 0.06825803, 0.006347..."
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.0,"[0.1988398, -0.27916974, 0.088556476, 0.002569..."
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.5,"[0.18002738, -0.32246664, 0.0646358, -0.026523..."


In [19]:
longformer_reviews.shape

(63230, 9)

### Save DataFrame

Put together dataframe and embeddings

In [20]:
%pwd

'/content/drive/MyDrive/NLP/Lab4'

In [21]:
%ls

bert_truncated.csv  Restaurant_reviews_R.csv


In [22]:
longformer_reviews.to_csv('longformer.csv', index=False)

## Section 3

### Read File

In [62]:
column_names = ['Rating', 'Date', 'Review', 'Business', 'City', 'State', 'Zip', 'Business Rating Score']
bert_split_reviews = pd.read_csv('Restaurant_reviews_R.csv', header=None, names=column_names)

In [63]:
bert_split_reviews.head()

,Rating,Date,Review,Business,City,State,Zip,Business Rating Score
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.0
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.5


In [64]:
bert_split_reviews.shape

(63230, 8)

### Import Model

Use fast tokenizer since it's faster, handles overflow, and is more memory efficient

In [42]:
from transformers import BertTokenizerFast, BertModel

In [43]:
import logging
# Set logging level to ERROR to mute informational messages
logging.getLogger("transformers").setLevel(logging.ERROR)

In [44]:
# Show time to run this
%%time
# Pre-trained fast tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
# Pre-trained model
model = BertModel.from_pretrained('bert-base-uncased')

CPU times: user 250 ms, sys: 424 ms, total: 674 ms
Wall time: 2.99 s


### Preprocessing (N/A)

Base BERT requires minimal preprocessing because BERT is trained to work with raw text.

Since we split and avg embeddings for reviews that are too long, we don't need to any preprocessing like truncation.

### Computation Setup

In [45]:
# Check if GPU is available and set computational device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [46]:
!nvidia-smi

Mon Mar 25 01:27:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0              31W /  70W |    663MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [47]:
# Move model to the selected device
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [48]:
# Enforce that the model is in evaluation mode
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

### Generate Embeddings
 - split review and average splits when review exceeds max size

Functions

In [49]:
def get_avg_embeddings_for_text (text, tokenizer, model, device):
    # input ids with overflowing tokens
    inputs = tokenizer(text,
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=512,
        stride=160,  # customize the stride to have more overlap
        return_overflowing_tokens=True
        )

    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    embeddings = []

    for i in range(input_ids.size(0)):
        with torch.no_grad():
            output = model(input_ids[i].unsqueeze(0), attention_mask=attention_mask[i].unsqueeze(0))
            segment_embedding = output.last_hidden_state.mean(dim=1).cpu().numpy()
            embeddings.append(segment_embedding)

    avg_embedding = np.mean(embeddings, axis=0)

    return avg_embedding

In [50]:
def get_all_embeddings (df, text_column, tokenizer, model, device, batch_size=32):
    all_embeddings = []

    # Iterate through in batches
    for i in tqdm(range(0, len(df), batch_size)):
        end_idx = i + batch_size
        batch_texts = df[text_column][i:end_idx]
        # Compute embeddings
        batch_embeddings = [get_avg_embeddings_for_text(text, tokenizer, model, device) for text in batch_texts]
        all_embeddings.extend(batch_embeddings)

    return np.vstack(all_embeddings)

Test this with one review longer than 512 (remember that row 25529 had length 1332 with truncation)

In [51]:
row = bert_split_reviews.iloc[25529,:]

In [52]:
review = row['Review']

In [53]:
print(review[:1000])  # this person seems very mad, no wonder they had the longest review

So I write this fact-based, detailed, FAIR-MINDED review of an establishment - a place that I SAID I want to try again, because I believe in supporting entrepreneurial chef-owners, gave it 3 stars (which means "A-OK" last time I checked the Yelp dictionary) - a review for which (thanks to fellow Yelpers' largesse) I got 5 'useful' and 3 'cool' (PLUS a private "excellent review" from a highly respected GOLD elite Yelper), and what do I get?  

I get a "Dear Paul" letter from an "Alex B." at Finesse, whose acerbic words do not exactly spell "customer-centric" even in the wildest interpretation of the term.  What is the MATTER with this world!!!!  I hope the owner and chef at Finesse knows you're doing this, Alex B., because, what you did when you wrote that hot-headed note, you're messing with his livelihood, his future, his restaurant's future, his FAMILY's future...

Weird thing is, for all your bombastic ways, what you UTTERLY failed to do in your tirade, Alex B., are A) acknowledge t

In [54]:
test_embeddings = get_avg_embeddings_for_text(review, tokenizer, model, device)

In [55]:
print(test_embeddings.shape)

(1, 768)


Do actual

In [56]:
# tqdm gives progress bars
from tqdm import tqdm

In [57]:
# Make embeddings in batches and observe time
%%time
embeddings = get_all_embeddings(bert_split_reviews, 'Review', tokenizer, model, device)

100%|██████████| 1976/1976 [15:37<00:00,  2.11it/s]


CPU times: user 15min 3s, sys: 3.75 s, total: 15min 7s
Wall time: 15min 37s


### Make Final DataFrame and Show

In [65]:
# Attach embeddings
bert_split_reviews['BERT Vectors'] = list(embeddings)

In [66]:
bert_split_reviews.head()

,Rating,Date,Review,Business,City,State,Zip,Business Rating Score,BERT Vectors
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,"[0.077761725, 0.17507069, 0.771844, -0.1101257..."
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,"[-0.039360136, 0.281392, 0.17292583, -0.040627..."
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,"[-0.012619782, -0.057393964, 0.27590153, 0.116..."
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.0,"[-0.14339037, -0.14566176, 0.4460344, 0.003198..."
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.5,"[0.042758364, -0.10380348, 0.15128294, 0.10330..."


In [67]:
bert_split_reviews.shape

(63230, 9)

### Save DataFrame

Put together dataframe and embeddings

In [68]:
%pwd

'/content/drive/MyDrive/NLP/Lab4'

In [69]:
%ls

bert_split.csv        bert_truncated.csv        longformer.csv      Restaurant_reviews_R.csv
bert_split_table.png  bert_truncated_table.png  longform_table.png


In [70]:
bert_split_reviews.to_csv('bert_split.csv', index=False)

## Section 4

### Read Files

In [8]:
bert_truncated_reviews = pd.read_csv('bert_truncated.csv')
longformer_reviews = pd.read_csv('longformer.csv')
bert_split_reviews = pd.read_csv('bert_split.csv')

In [9]:
bert_truncated_reviews.head()

,Rating,Date,Review,Business,City,State,Zip,Business Rating Score,Base BERT Vectors
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,[-0.66720617 -0.5012724 -0.97912174 0.697637...
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,[-5.88722348e-01 -5.28097510e-01 -9.80367780e-...
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,[-0.854242 -0.44665197 -0.9462228 0.712074...
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.0,[-7.29353011e-01 -2.95089036e-01 -8.91691446e-...
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.5,[-0.7442665 -0.47785082 -0.988199 0.684198...


In [10]:
longformer_reviews.head()

,Rating,Date,Review,Business,City,State,Zip,Business Rating Score,Longform Vectors
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,[ 1.84318155e-01 -2.46027768e-01 5.89351840e-...
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,[ 1.82556719e-01 -3.22503984e-01 8.60914811e-...
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,[ 1.88996345e-01 -3.06773067e-01 6.82580322e-...
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.0,[ 0.1988398 -0.27916974 0.08855648 0.002569...
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.5,[ 1.80027381e-01 -3.22466642e-01 6.46357983e-...


In [11]:
bert_split_reviews.head()

,Rating,Date,Review,Business,City,State,Zip,Business Rating Score,BERT Vectors
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,[ 7.77617246e-02 1.75070688e-01 7.71844029e-...
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,[-3.93601358e-02 2.81392008e-01 1.72925830e-...
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,[-1.26197822e-02 -5.73939644e-02 2.75901526e-...
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.0,[-1.43390372e-01 -1.45661756e-01 4.46034402e-...
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.5,[ 4.27583642e-02 -1.03803478e-01 1.51282936e-...


In [12]:
print(bert_truncated_reviews.shape)
print(longformer_reviews.shape)
print(bert_split_reviews.shape)

(63230, 9)
(63230, 9)
(63230, 9)


### Make DataFrame and Show

Make collected DataFrame with all embeddings

In [13]:
# Concatenate the dataframes
combined_reviews = pd.concat([bert_truncated_reviews, longformer_reviews['Longform Vectors'], bert_split_reviews['BERT Vectors']], axis=1)
combined_reviews.head()

,Rating,Date,Review,Business,City,State,Zip,Business Rating Score,Base BERT Vectors,Longform Vectors,BERT Vectors
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.5,[-0.66720617 -0.5012724 -0.97912174 0.697637...,[ 1.84318155e-01 -2.46027768e-01 5.89351840e-...,[ 7.77617246e-02 1.75070688e-01 7.71844029e-...
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.0,[-5.88722348e-01 -5.28097510e-01 -9.80367780e-...,[ 1.82556719e-01 -3.22503984e-01 8.60914811e-...,[-3.93601358e-02 2.81392008e-01 1.72925830e-...
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.5,[-0.854242 -0.44665197 -0.9462228 0.712074...,[ 1.88996345e-01 -3.06773067e-01 6.82580322e-...,[-1.26197822e-02 -5.73939644e-02 2.75901526e-...
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.0,[-7.29353011e-01 -2.95089036e-01 -8.91691446e-...,[ 0.1988398 -0.27916974 0.08855648 0.002569...,[-1.43390372e-01 -1.45661756e-01 4.46034402e-...
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.5,[-0.7442665 -0.47785082 -0.988199 0.684198...,[ 1.80027381e-01 -3.22466642e-01 6.46357983e-...,[ 4.27583642e-02 -1.03803478e-01 1.51282936e-...


Make sure that the embedding vectors are arrays (they get accidentally made into strings when saving/reading)

Innefficent but happens when we want to have the whole embedding in one column. Alternatively, we could also just use a pkl file, but the assignment recommends csv

In [14]:
import ast

def parse_vector(vec_str):
    # Clean the string and manually parse it
    clean_str = vec_str.strip("[] ")
    # Split the string into elements given space separation, and convert each to float
    elements = clean_str.split()
    try:
        return np.array([float(elem) for elem in elements])
    except ValueError:
        # Log or handle cases where conversion fails
        print(f"Conversion failed for: {vec_str}")
        return np.nan  # Return NaN or appropriate value for failed conversions

In [15]:
# Parse and make array
columns_to_parse = ['Base BERT Vectors', 'Longform Vectors', 'BERT Vectors']

for column in columns_to_parse:
    combined_reviews[column] = combined_reviews[column].apply(parse_vector)

In [16]:
# Check the type of the first element to confirm conversion
print(type(combined_reviews.loc[0, 'Base BERT Vectors']))
print(type(combined_reviews.loc[0, 'Longform Vectors']))
print(type(combined_reviews.loc[0, 'BERT Vectors']))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


Check some sizes

In [23]:
row = combined_reviews.iloc[0]
base = row['Base BERT Vectors']
longform = row['Longform Vectors']
bert = row['BERT Vectors']

In [24]:
print(base.shape, base[0].shape)
print(longform.shape, longform[0].shape)
print(bert.shape, bert[0].shape)

(768,) ()
(768,) ()
(768,) ()


Group and average

In [25]:
# Function to average embeddings when doing grouping
def mean_embedding(embedding_lists):
    embeddings_array = np.array(embedding_lists)  # Convert the list of embeddings to a numpy array for easier calculation
    mean_embedding = np.mean(embeddings_array, axis=0)  # 0-axis for column-wise mean
    return mean_embedding.tolist()  # Convert back to list

In [27]:
# Group reviews by business
aggregated_reviews = combined_reviews.groupby(['Business', 'City', 'State', 'Zip']).agg({
    'Business Rating Score': 'mean',
    'Review': lambda reviews: ' '.join(reviews),
    'Base BERT Vectors': mean_embedding,
    'Longform Vectors': mean_embedding,
    'BERT Vectors': mean_embedding
}).reset_index()

In [28]:
aggregated_reviews.head()

,Business,City,State,Zip,Business Rating Score,Review,Base BERT Vectors,Longform Vectors,BERT Vectors
0,1/2 Roll,Vancouver,BC,V5V 3P6,3.0,The ambience is kind of funky & interesting. F...,"[-0.634989940117647, -0.43836614217647063, -0....","[0.16952809594117646, -0.29044892414705886, 0....","[-0.014949210241764706, -0.07912803805823526, ..."
1,21 Nickels,Watertown,MA,02472,4.0,I like 21 Nickels a ton and they'd attract mor...,"[-0.5937912840474138, -0.38311916175689664, -0...","[0.1805416628103448, -0.2907828731206898, 0.07...","[0.10340431334413792, 0.06072802121982758, 0.2..."
2,24 Hour Pizza Delivery,Boston,MA,02124,1.0,This place needs to be shut down. I couldn't b...,"[-0.5929443752424243, -0.45256393472727274, -0...","[0.16708752587878792, -0.2828345722121212, 0.0...","[0.15056832087151512, 0.05912671888245454, 0.3..."
3,3 Monkeys Pub & Grill,Vancouver,WA,98665,3.0,"Well, it's re-opened.\n\nStopped in today just...","[-0.6058024895489798, -0.39999759583673467, -0...","[0.18460252002040817, -0.2875949916734694, 0.0...","[0.10219423699734692, 0.023166088148163268, 0...."
4,5th St Ribs-N-Blues,Atlanta,GA,30308,3.0,First visit here for my weekly southern BBQ fi...,"[-0.6139719890882354, -0.420025550264706, -0.9...","[0.17346425955882355, -0.2965336888235295, 0.0...","[0.08408000523058823, 0.008251015441470593, 0...."


In [35]:
# Average over all representations as well (this assumes that these different
# models have similar embedding spaces, which may be reasonable given that they
# are all based on BERT)

def average_embeddings(row):
    embeddings = [row[col] for col in ['Base BERT Vectors', 'Longform Vectors', 'BERT Vectors']]
    # Stack embeddings along a new axis and calculate the mean across that axis
    return np.mean(np.stack(embeddings), axis=0)

aggregated_reviews['Averaged Embeddings'] = aggregated_reviews.apply(average_embeddings, axis=1)

Show

In [36]:
aggregated_reviews.head()

,Business,City,State,Zip,Business Rating Score,Review,Base BERT Vectors,Longform Vectors,BERT Vectors,Averaged Embeddings
0,1/2 Roll,Vancouver,BC,V5V 3P6,3.0,The ambience is kind of funky & interesting. F...,"[-0.634989940117647, -0.43836614217647063, -0....","[0.16952809594117646, -0.29044892414705886, 0....","[-0.014949210241764706, -0.07912803805823526, ...","[-0.16013701813941175, -0.26931436812725495, -..."
1,21 Nickels,Watertown,MA,02472,4.0,I like 21 Nickels a ton and they'd attract mor...,"[-0.5937912840474138, -0.38311916175689664, -0...","[0.1805416628103448, -0.2907828731206898, 0.07...","[0.10340431334413792, 0.06072802121982758, 0.2...","[-0.1032817692976437, -0.2043913378859196, -0...."
2,24 Hour Pizza Delivery,Boston,MA,02124,1.0,This place needs to be shut down. I couldn't b...,"[-0.5929443752424243, -0.45256393472727274, -0...","[0.16708752587878792, -0.2828345722121212, 0.0...","[0.15056832087151512, 0.05912671888245454, 0.3...","[-0.09176284283070707, -0.22542392935231315, -..."
3,3 Monkeys Pub & Grill,Vancouver,WA,98665,3.0,"Well, it's re-opened.\n\nStopped in today just...","[-0.6058024895489798, -0.39999759583673467, -0...","[0.18460252002040817, -0.2875949916734694, 0.0...","[0.10219423699734692, 0.023166088148163268, 0....","[-0.10633524417707489, -0.22147549978734693, -..."
4,5th St Ribs-N-Blues,Atlanta,GA,30308,3.0,First visit here for my weekly southern BBQ fi...,"[-0.6139719890882354, -0.420025550264706, -0.9...","[0.17346425955882355, -0.2965336888235295, 0.0...","[0.08408000523058823, 0.008251015441470593, 0....","[-0.11880924143294119, -0.2361027412155883, -0..."


In [40]:
aggregated_reviews.shape

(1075, 10)

### Save DataFrame

In [37]:
%pwd

'/content/drive/MyDrive/NLP/Lab4'

In [38]:
%ls

bert_split.csv        bert_truncated.csv        longformer.csv      Restaurant_reviews_R.csv
bert_split_table.png  bert_truncated_table.png  longform_table.png


In [39]:
aggregated_reviews.to_csv('aggregated.csv', index=False)